# Тестовое задание на позицию Data Analyst - Бурый Артём
<div style="text-align: right"><a target="_blank" href="https://drive.google.com/file/d/14Hl1gFeVGPm1ChDI1HlNQB4Tz6bFH1Qd/view?usp=sharing">Резюме</a></div>

## SQL блок
Для начала создадим необходимые для задания таблицы

```sql
create table registration ("user_id" int not null primary key,
							"timestamp" timestamp,
							"source" varchar,
							"cost" int);
						
insert into registration ("user_id", "timestamp", "source", "cost") 
	values (generate_series(1, 1000), 
		timestamp '2023-01-01 00:00:00' + random() * interval '300 days',
		(array['c1', 'c2', 'c3'])[floor(random() * 3 + 1)],
		floor(random() * 100 + 1));
	
create table payments ("user_id" int not null,
							"payment_id" int not null primary key,
							"payment_ts" timestamp,
							"payment_usd" int,
							constraint fk_user
						      foreign key(user_id) 
							  	references registration(user_id));

insert into payments ("user_id", "payment_id", "payment_ts", "payment_usd")
select t.user_id,
	t.payment_id,
	t.timestamp,
	t.payment_usd 
from (select floor(random()*1000 + 1) as "user_id",
	generate_series(1,2000) as "payment_id",
	timestamp '2023-01-01 00:00:00' + random() * interval '600 days' as "timestamp",
	floor(random()*100 + 1) as "payment_usd"
) as t
left join registration r on r.user_id = t.user_id
where r.timestamp < t.timestamp
	and t.timestamp < now();
```

1. Конверсію з реєстрації в оплату за місяцями реєстрації ( Інвестори прийшли -і терміново хочуть знати динаміку цього показнику через загальну рецесію )
```sql
select to_char(r."timestamp", 'YYYY-MM') as "year-month",
	1.0 * sum(case when r.user_id in (select distinct p.user_id from payments p) then 1 else 0 end) / nullif(count(*),0) as "conversion"
from registration r 
group by to_char(r."timestamp", 'YYYY-MM')
order by to_char(r."timestamp", 'YYYY-MM');
```
2. Середній LTV клієнтів 180 днів за сорсами трафіку. LTV вважати суму всіх платежів. (Маркетологам дуже треба це знання для оптимізації трафіку в рамках ROMI + з якої ставки стартувати в нових партнерках)
```sql
TBA
```
3. Типовий час між 1ою та 2ою оплатою ( Хочеться дивитися на конверсію з 1ої в 2гу оплату у звіті -але останні дні завжди мають нижчі показники -то ж треба зафіксувати період , в який дивимося  )
```sql
with t as (
select p.user_id,
	row_number () OVER (partition by p.user_id	order by p.payment_ts),
	lead(p.payment_ts, 1) OVER (partition by p.user_id	order by p.payment_ts) - p.payment_ts as time_delta
from payments p
)
select avg(t.time_delta) as average_time_delta
from t
where t.row_number = 1
	and t.time_delta is not null;
```
На созданных таблицах данные запросы отрабатывали без ошибок и выдавали адекватные результаты.
## Data Exploration блок

In [1]:
import pandas as pd

In [8]:
url = "https://docs.google.com/spreadsheets/d/1rLg0ZvTIAESz2R6zEc3AGjL1sdRMj4Zl-55PtukyTl0/export?format=xlsx"
df = pd.read_excel(url, sheet_name="Exploration data")

In [9]:
df

,user_id,registration_date,is_test_user,is_high_value_customer,paid_7d,paid_8_14d,payments_7d,payments_8_14d,first_payment_date,last_payment_date,...,open_8_14d,click_8_14d,page_visits_7d,page_visits_8_14d,logins_7d,logins_8_14d,first_session_duration_minutes,user_first_os,score1,score2
0,442950.0,2022-03-06 08:39:10.000000,0.0,0.0,0.000000,0.0,0.0,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,11.0,Android,6.0,17.0
1,957150.0,2022-06-17 03:17:09.000000,0.0,1.0,8.102620,0.0,2.0,0.0,2022-06-17 05:41:12.000000,2022-06-18 21:52:55.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,Android,48.0,17.0
2,1112646.0,2022-07-16 20:13:52.000000,0.0,0.0,10.017636,0.0,2.0,0.0,2022-07-21 14:36:35.000000,2022-07-21 16:43:25.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,Android,10.0,63.0
3,1294040.0,2022-08-14 15:29:50.000000,0.0,0.0,2.283319,0.0,1.0,0.0,2022-08-16 10:39:04.000000,2022-08-16 10:39:04.000000,...,13.0,4.0,0.0,0.0,0.0,0.0,4.0,iOS,76.0,58.0
4,1414332.0,2022-09-06 17:41:25.000000,0.0,0.0,8.333930,0.0,1.0,0.0,2022-09-06 17:45:37.000000,2022-09-06 17:45:37.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,Android,44.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,1071006.0,2022-07-10 03:12:41.000000,0.0,0.0,4.373870,0.0,1.0,0.0,2022-07-10 03:18:20.000000,2022-07-10 03:18:20.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,15.0,iOS,72.0,9.0
4996,847845.0,2022-05-22 01:53:13.000000,0.0,0.0,0.000000,0.0,0.0,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,13.0,Android,94.0,7.0
4997,153552.0,2022-01-24 07:49:51.000000,0.0,0.0,0.000000,0.0,0.0,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,Android,27.0,28.0
4998,171984.0,2022-01-26 14:02:40.000000,0.0,0.0,17.891515,0.0,1.0,0.0,2022-01-28 11:54:23.000000,2022-01-28 11:54:23.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,Android,2.0,18.0
